In [1]:
import os
import time
import random

import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import pickle

In [2]:
images_dir = './PetImages'
categories = ['Dog', 'Cat']
img_size = 70

# Preprocessing

In [3]:
training_data = []

def create_training_data():    
    for category in categories:
        path = os.path.join(images_dir, category)
        class_num = categories.index(category) # dog == 0, cat == 1
        for img in os.listdir(path):
            try:
                # using grayscale because color probably doesn't help us here
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_img_arr = cv2.resize(img_arr, (img_size, img_size))
                training_data.append([resized_img_arr, class_num])
            except Exception as e:
                print(e)
            
create_training_data()


OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\

In [4]:
print(len(training_data))

24946


In [5]:
random.shuffle(training_data)
for sample in training_data[:5]:
    print(sample[1])

0
1
1
1
0


In [6]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
# for some reason regular lists dont work so we convert to np arrays
X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

In [7]:
# pickle dump to save for later use
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [11]:
print(f'running on {'gpu' if len(tf.config.list_physical_devices('GPU')) > 0 else 'cpu'}')

running on gpu


# Model optimization
## Test different dense layers, layer sizes, conv layers

In [ ]:
dense_layer_options = [0, 1, 2]
layer_size_options = [32, 64, 128]
conv_layer_options = [1, 2, 3]


for dense_layer in dense_layer_options:
    for layer_size in layer_size_options:
        for conv_layer in conv_layer_options:
            model_name = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
            tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))

            optimized_model = Sequential()
            
            optimized_model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            optimized_model.add(Activation("relu"))
            optimized_model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                optimized_model.add(Conv2D(layer_size, (3,3)))
                optimized_model.add(Activation("relu"))
                optimized_model.add(MaxPooling2D(pool_size=(2,2)))

            optimized_model.add(Flatten())
            
            for l in range(dense_layer):
                optimized_model.add(Dense(layer_size))
                optimized_model.add(Activation("relu"))

            optimized_model.add(Dense(1))
            optimized_model.add(Activation("sigmoid"))

            optimized_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
            optimized_model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

## Result: 3 conv 0 dense 128 layer size had least eval loss

## Try higher layer size options

In [13]:
layer_size_options = [256, 512, 1024]


for layer_size in layer_size_options:
    model_name = f"3-conv-{layer_size}-nodes-0-dense-{int(time.time())}"
    tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))

    optimized_model = Sequential()

    optimized_model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Conv2D(layer_size, (3,3)))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Conv2D(layer_size, (3,3)))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Flatten())

    optimized_model.add(Dense(1))
    optimized_model.add(Activation("sigmoid"))

    optimized_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    optimized_model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

Epoch 1/10
135/702 [====>.........................] - ETA: 14s - loss: 0.6894 - accuracy: 0.5194

KeyboardInterrupt: 

## Final results: 3 conv layers 64 layer size 0 dense

# Final model training

In [14]:
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

X = X / 255.0
tensorboard = TensorBoard(log_dir="logs/{3-conv-64-nodes-0-dense}")
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

model.save('64x3-CNN.h5')

Epoch 1/10
702/702 [==============================] - 7s 9ms/step - loss: 0.6150 - accuracy: 0.6533 - val_loss: 0.5851 - val_accuracy: 0.7006
Epoch 2/10
702/702 [==============================] - 5s 8ms/step - loss: 0.4992 - accuracy: 0.7585 - val_loss: 0.5038 - val_accuracy: 0.7403
Epoch 3/10
702/702 [==============================] - 5s 7ms/step - loss: 0.4467 - accuracy: 0.7919 - val_loss: 0.4563 - val_accuracy: 0.7852
Epoch 4/10
702/702 [==============================] - 5s 7ms/step - loss: 0.4050 - accuracy: 0.8184 - val_loss: 0.4268 - val_accuracy: 0.8012
Epoch 5/10
702/702 [==============================] - 5s 7ms/step - loss: 0.3647 - accuracy: 0.8364 - val_loss: 0.4046 - val_accuracy: 0.8156
Epoch 6/10
702/702 [==============================] - 5s 7ms/step - loss: 0.3313 - accuracy: 0.8547 - val_loss: 0.4063 - val_accuracy: 0.8172
Epoch 7/10
702/702 [==============================] - 5s 7ms/step - loss: 0.3060 - accuracy: 0.8652 - val_loss: 0.3785 - val_accuracy: 0.8313
Epoch 